<a href="https://colab.research.google.com/github/Abdalmughith/machine-learning-projects/blob/master/STARGARDT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**USING ARTIFICIAL INTELLIGENCE FOR DETECTION OF STARGARDT’S DISEASE FUNDUS PHOTOS**

***Purpose.*** Formulating an artificial intelligence model to differentiate fundus photos with Stargardt’s disease from normal fundus photos.
 
***Methods***. Fundus photos of eyes with established diagnosis of Stargardt’s disease (n=25) and normal fundi (n=25) were used to train a TenserFlow Google Cloud based Convolutional Neural Network (CNN) containing three convolutional layers followed by two fully connected layer ending with sigmoid activation function. The numeric values of the sigmoid were converted to a percentage value with the smallest corresponding to 0% and the highest to 100%. Values >50% were considered diagnostic for Stargardt’s disease. 
.
 
***Results.*** The model was tested with fundus photos that were not used for the training of the model. The model was able to diagnose correctly fundus photos with Stargardt’s disease (n=5), (values ranging 60.5-93.9%) vs normal fundi (n=10), (values ranging 15.7-35.3%), (100% specificity and 100% sensitivity).
 
***Conclusions***:The deep learning artificial intelligence model was able to diagnose Stargardt’s disease depending on fundus images with 100% specificity and 100% sensitivity.


In [0]:
import os 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import numpy as np
from numba import jit
import os

import math
import csv
from sklearn import preprocessing
import cv2
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
img_width, img_height = 615, 800

flags = tf.app.flags
flags.DEFINE_string('image', '', 'Path to image')
FLAGS = flags.FLAGS


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [0]:



def standard_deviation_image(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_output = clahe.apply(image)
    result = clahe_output.copy()
    i = 0
    j = 0
    while i < image.shape[0]:
        j = 0
        while j < image.shape[1]:
            sub_image = clahe_output[i:i + 20, j:j + 25]
            var = np.var(sub_image)
            result[i:i + 20, j:j + 25] = var
            j = j + 25
        i = i + 20
    return result




def get_average_intensity(green_channel):
    average_intensity = green_channel.copy()
    i = 0
    j = 0
    while i < green_channel.shape[0]:
        j = 0
        while j < green_channel.shape[1]:
            sub_image = green_channel[i:i + 20, j:j + 25]
            mean = np.mean(sub_image)
            average_intensity[i:i + 20, j:j + 25] = mean
            j = j + 25
        i = i + 20
    result = np.reshape(average_intensity, (average_intensity.size, 1))
    return result


def get_average_hue(hue_image):
    average_hue = hue_image.copy()
    i = 0
    j = 0
    while i < hue_image.shape[0]:
        j = 0
        while j < hue_image.shape[1]:
            sub_image = hue_image[i:i + 20, j:j + 25]
            mean = np.mean(sub_image)
            average_hue[i:i + 20, j:j + 25] = mean
            j = j + 25
        i = i + 20
    result = np.reshape(average_hue, (average_hue.size, 1))
    return result


def get_average_saturation(hue_image):
    average_hue = hue_image.copy()
    i = 0
    j = 0
    while i < hue_image.shape[0]:
        j = 0
        while j < hue_image.shape[1]:
            sub_image = hue_image[i:i + 20, j:j + 25]
            mean = np.mean(sub_image)
            average_hue[i:i + 20, j:j + 25] = mean
            j = j + 25
        i = i + 20
    result = np.reshape(average_hue, (average_hue.size, 1))
    return result


def edge_pixel_image(image, bv_image):
    edge_result = image.copy()
    edge_result = cv2.Canny(edge_result, 30, 100)
    cv2.imwrite('Canny-over-original.jpg', edge_result)
    i = 0
    j = 0
    while i < image.shape[0]:
        j = 0
        while j < image.shape[1]:
            if edge_result[i, j] == 255 and bv_image[i, j] == 255:
                edge_result[i, j] = 0
            j = j + 1
        i = i + 1

    newfin = cv2.dilate(edge_result, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=1)
    return newfin


def extract_bv(image):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    contrast_enhanced_green_fundus = clahe.apply(image)
    # applying alternate sequential filtering (3 times closing opening)
    r1 = cv2.morphologyEx(contrast_enhanced_green_fundus, cv2.MORPH_OPEN,
                          cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)), iterations=1)
    cv2.imwrite('extract_bv-r1-morphology-opening.jpg', r1)
    R1 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)), iterations=1)
    cv2.imwrite('extract_bv-R1-morphology-opening.jpg', R1)
    r2 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11)), iterations=1)
    cv2.imwrite('extract_bv-r2-morphology-opening.jpg', r2)
    R2 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11)), iterations=1)
    cv2.imwrite('extract_bv-R2-morphology-opening.jpg', R2)
    r3 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (23, 23)), iterations=1)
    cv2.imwrite('extract_bv-r3-morphology-opening.jpg', r3)
    R3 = cv2.morphologyEx(r3, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (23, 23)), iterations=1)
    cv2.imwrite('extract_bv-R3-morphology-opening.jpg', R3)
    f4 = cv2.subtract(R3, contrast_enhanced_green_fundus)
    cv2.imwrite('extract_bv_subtractfrom_ogienal.jpg', f4)
    f5 = clahe.apply(f4)
    cv2.imwrite('extract_bv_subtractfrom_ogienal-histogram.jpg', f5)
    # removing very small contours through area parameter noise removal
    ret, f6 = cv2.threshold(f5, 15, 255, cv2.THRESH_BINARY)
    mask = np.ones(f5.shape[:2], dtype="uint8") * 255
    im2, contours, hierarchy = cv2.findContours(f6.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) <= 200:
            cv2.drawContours(mask, [cnt], -1, 0, -1)
    im = cv2.bitwise_and(f5, f5, mask=mask)
    ret, fin = cv2.threshold(im, 15, 255, cv2.THRESH_BINARY_INV)
    newfin = cv2.erode(fin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=1)
    cv2.imwrite('extract_bv_remove_small_contours.jpg', newfin)
    # removing blobs of microaneurysm & unwanted bigger chunks taking in consideration they are not straight lines like blood
    # vessels and also in an interval of area
    fundus_eroded = cv2.bitwise_not(newfin)
    xmask = np.ones(image.shape[:2], dtype="uint8") * 255
    x1, xcontours, xhierarchy = cv2.findContours(fundus_eroded.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in xcontours:
        shape = "unidentified"
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * peri, False)
        if len(approx) > 4 and cv2.contourArea(cnt) <= 3000 and cv2.contourArea(cnt) >= 100:
            shape = "circle"
        else:
            shape = "veins"
        if (shape == "circle"):
            cv2.drawContours(xmask, [cnt], -1, 0, -1)

    finimage = cv2.bitwise_and(fundus_eroded, fundus_eroded, mask=xmask)
    blood_vessels = cv2.bitwise_not(finimage)
    dilated = cv2.erode(blood_vessels, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)), iterations=1)
    cv2.imwrite('extract_bv_remove_big_contours.jpg', dilated)
    # dilated1 = cv2.dilate(blood_vessels, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)), iterations=1)
    blood_vessels_1 = cv2.bitwise_not(dilated)
    return blood_vessels_1
def predict(image2predict):
    model1 = Sequential()
    model1.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))
    model1.add(Conv2D(32, (3, 3)))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))
    model1.add(Conv2D(64, (3, 3)))
    model1.add(Activation('relu'))
    model1.add(MaxPooling2D(pool_size=(2, 2)))
    model1.add(Flatten())
    model1.add(Dense(64))
    model1.add(Activation('relu'))
    model1.add(Dropout(0.5))
    model1.add(Dense(1))
    model1.add(Activation('sigmoid'))
    model1.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
    model1.load_weights('model.h5')
    fundus1 = cv2.imread(image2predict)
    fundus = cv2.resize(fundus1, (800, 615))
    fundus_mask = cv2.imread("MASK.bmp")
    fundus_mask = cv2.resize(fundus_mask, (800, 615))
    f1 = cv2.bitwise_and(fundus[:, :, 0], fundus_mask[:, :, 0])
    f2 = cv2.bitwise_and(fundus[:, :, 1], fundus_mask[:, :, 1])
    f3 = cv2.bitwise_and(fundus[:, :, 2], fundus_mask[:, :, 2])
    fundus_dash = cv2.merge((f1, f2, f3))
    cv2.imwrite('1-masking.jpg', fundus_dash)

    b, g, r = cv2.split(fundus_dash)
    hsv_fundus = cv2.cvtColor(fundus_dash, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_fundus)
    gray_scale = cv2.cvtColor(fundus_dash, cv2.COLOR_BGR2GRAY)
    cv2.imwrite('2-gray_scale.jpg', gray_scale)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    contrast_enhanced_fundus = clahe.apply(gray_scale)
    cv2.imwrite('3-histogram-gray_scale.jpg', contrast_enhanced_fundus)
    contrast_enhanced_green_fundus = clahe.apply(g)
    cv2.imwrite('4-contrast_enhanced_green_fundus.jpg', contrast_enhanced_green_fundus)

    average_intensity = get_average_intensity(contrast_enhanced_green_fundus) / 255
    average_hue = get_average_hue(h) / 255
    average_saturation = get_average_saturation(s) / 255

    bv_image_dash = extract_bv(g)
    cv2.imwrite('5-green-extract_bv.jpg', bv_image_dash)
    bv_image = extract_bv(gray_scale)
    cv2.imwrite('6-gray_scale-extract_bv.jpg', bv_image)

    var_fundus = standard_deviation_image(contrast_enhanced_fundus)

    edge_feature_output = edge_pixel_image(gray_scale, bv_image)
    cv2.imwrite('7-removeUnionBetweenCanny-over-originaland.jpgBv_image.jpg', edge_feature_output)
    newfin = cv2.dilate(edge_feature_output, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)), iterations=1)
    cv2.imwrite('8-dilateWithMORPH_ELLIPSE.jpg', newfin)
    edge_candidates = cv2.erode(newfin, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=1)

    edge_candidates = np.uint8(edge_candidates)
    cv2.imwrite('9-ErosionWithMORPH_ELLIPSE.jpg', edge_candidates)

    cv2.imwrite('10.jpg', edge_candidates)
    img = cv2.imread('10.jpg')
    img = np.reshape(img,[1,615,800,3])
    classes = model1.predict_classes(img)
    if (classes[0][0]==1):
        print('yes')
    else:
        print('no')